<a href="https://colab.research.google.com/github/OculusMode/ILL/blob/main/experimental/Subspaces_with_ILL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import random
from numpy import linalg as LA

In [4]:
N = 10
M = 4
TOTAL_PARTITIONS = 100

In [59]:
class Subspace:
  def __init__(self, basis, indices):
    self.basis = basis
    self.indices = indices

class SubspacePartition:
  def __init__(self, subspaces, N=10):
    self.subspaces = subspaces # list of Subspace class
    self.N = N
  def is_coarser(self, other_partn):
    """Returns 1 if coarse, -1 if finer, 0 if undecidable"""
    basis_1 = [i.indices for i in self.subspaces]
    basis_2 = [i.indices for i in other_partn.subspaces]
    l_1 = len(basis_1)
    l_2 = len(basis_2)
    # print(basis_1, basis_2)
    # if both lengths are same then they cannot be finer/coarser
    if l_1 == l_2:
      return 0
    # the rule with smaller rule_domain MAY BE coarser
    if l_1 > l_2:
      for i in basis_1:
        for j in basis_2:
          if not (set(i).issubset(set(j)) or set(i).isdisjoint(set(j))):
            return 0
      return -1
    else:
      for i in basis_2:
        for j in basis_1:
          if not (set(i).issubset(set(j)) or set(i).isdisjoint(set(j))):
            return 0
      return 1
  def print_partition(self):
    for S in self.subspaces:
      print(S.basis, S.indices)
    
  def print_basis_idx(self):
    for S in self.subspaces:
      print(S.indices, end=' ')
    print()

def get_random_bases(N):
  while True:
    random_matrix = np.random.random((N, N))
    random_basis, _ = LA.qr(random_matrix)
    # making sure we got N basis
    if LA.matrix_rank(random_basis) == N:
      break
  return random_basis


def get_random_vector(N, is_normalized = True):
  v = random.random(N)
  if is_normalized:
    return v/LA.norm(v)
  return v


def get_random_subspace_indexes(N, M):
  # putting N-2 since we want maximum number as N-1
  # taking number M-1 times since we will use last left out portion for last partition
  split_points = np.random.choice(N - 2, M - 1, replace=False) + 1
  split_points.sort()
  data = np.arange(N)
  random.shuffle(data)
  random_subspace_indexes = np.split(data, split_points)
  return random_subspace_indexes


def make_partition(basis_matrix, N, M):
  # given an orthonormal basis matrix, give M random subpaces which have no basis in common
  indexes = get_random_subspace_indexes(N, M)
  # return [(basis_matrix[i], i) for i in indexes]
  return SubspacePartition(tuple(Subspace(basis_matrix[indice], indice) for indice in indexes))


def make_random_partitions(basis_matrix, total_partitions, N):
  mean = N//2
  std = N/8
  # I am using normal distribution for this, since we will have more possible number of partititons on the middle side.
  s = np.rint(random.normal(mean, std, total_partitions))
  """
  # WE CAN DO THIS TO SEE OUR DISTRIBUTION OVER HISTOGRAM
  import matplotlib.pyplot as plt
  num_bins = N//2
  n, bins, patches = plt.hist(s, num_bins)
  plt.show()
  """
  for i in s:
    if 0 < i< N:
      yield make_partition(basis_matrix, N, M=int(i))

random_basis = get_random_bases(N)
# for i in make_random_partitions(random_basis, TOTAL_PARTITIONS, N):
  # i.print_partition()
  # i.print_basis_idx()
# get_random_vector(10)
# make_partition(random_basis, N, M)

random_partitions = [i for i in make_random_partitions(random_basis, TOTAL_PARTITIONS, N)]

# p1 = random_partitions[0]
# p2 = random_partitions[1]
for idx, i in enumerate(random_partitions):
  for j in random_partitions[idx:]:
    k = i.is_coarser(j)
    if k != 0:
      i.print_basis_idx()
      j.print_basis_idx()
      print(k, sep='  =>   ')

[0] [3] [9 8 4] [5] [6 7] [2 1] 
[9 8 4] [3] [2 5 1 0] [6 7] 
-1
[3] [7 8 5] [9 1 2 4] [6 0] 
[3] [0] [9] [4 1 2] [6] [8 7 5] 
1
[5] [9 0 2 8 1] [6 4 7 3] 
[3] [5] [1] [2] [9] [4] [8 0] [6 7] 
1
[2] [5] [1] [6 7 3 8 0 4 9] 
[0 4] [5] [8] [2] [1] [9 3] [7 6] 
1
[2] [5] [1] [6 7 3 8 0 4 9] 
[3] [5] [1] [2] [9] [4] [8 0] [6 7] 
1
[1 2 8 0] [4 3 9] [5 7 6] 
[3] [5] [1] [2] [9] [4] [8 0] [6 7] 
1
[2] [7] [1 6 3 5 4 9] [8 0] 
[3] [1] [8 0] [4 6] [2] [7] [5 9] 
1
[0] [2] [7 8] [4] [6] [9 1] [5 3] 
[2 7 8] [6 1 4 0 9] [3 5] 
-1
[9 5] [4] [2] [7 6] [3] [8 1 0] 
[3] [5] [1] [2] [9] [4] [8 0] [6 7] 
1
[0 1] [2 3] [8 5 9 4] [6 7] 
[2] [9] [0 1] [7 6] [3] [5] [8 4] 
1
[9 7] [0] [8] [1 3] [2] [4 6 5] 
[5 8 4 6 1 3 0] [7 9 2] 
-1
[5 8 4 6 1 3 0] [7 9 2] 
[6] [8] [7] [1] [5 3 4] [0] [2 9] 
1
[2 1 7] [8] [4 5 0 3 6 9] 
[6] [9 5 3] [7] [0] [8] [4] [1 2] 
1
[2 1 7] [8] [4 5 0 3 6 9] 
[7] [8] [6 5] [4] [9] [0 3] [1 2] 
1
[9 8 7 1 2 4] [5 0] [3 6] 
[7 4] [0 5] [1] [6] [8] [3] [2 9] 
1
[9 8 7 1 2 4] [5 0] [

In [ ]:
# TO TEST THE IS_COARSER
# subspaces_2 = [
#   Subspace([np.ones(10)], [0]),
#   Subspace([np.ones(10), np.ones(10)], [1,2]),
#   Subspace([np.ones(10), np.ones(10)], [3,4]),
#   Subspace([np.ones(10), np.ones(10)], [5,6]),
#   Subspace([np.ones(10), np.ones(10)], [7,8]),
#   Subspace([np.ones(10), np.ones(10)], [9]),
# ]
# subspaces_1 = [
#   Subspace([np.ones(10)], [0,1,2,3,4,5,6]),
#   Subspace([np.ones(10)], [7,8, 9]),
# ]
# p1 = SubspacePartition(subspaces_2)
# p2 = SubspacePartition(subspaces_1)
# p1.print_basis_idx()
# p2.print_basis_idx()
# print(p1.is_coarser(p2))